In [25]:
import re
import heapq

def parse_logic(text: str):
    """
    Parse a logical description in the custom DSL into:
      - entity (str)
      - facts (set of atoms)
      - rules (list of (premises tuple, conclusion str))
      - query (str)
    
    DSL tokens:
      [PERIOD], [IMPLY], [AND], "query : X is Y"
    """
    # Split on [PERIOD] and strip
    clauses = [c.strip() for c in re.split(r'\[PERIOD\]', text) if c.strip()]
    facts = set()
    rules = []
    query = None
    entity = None
    
    for clause in clauses:
        # Query
        if clause.lower().startswith('query'):
            m = re.match(r'query\s*:\s*(\w+)\s+is\s+(\w+)', clause, re.IGNORECASE)
            if m:
                ent, atom = m.group(1).lower(), m.group(2).lower()
                query = atom
                entity = ent
        # Rule
        elif '[IMPLY]' in clause:
            lhs, rhs = clause.split('[IMPLY]')
            # premises
            lhs = lhs.strip()
            if '[AND]' in lhs:
                premises = tuple(p.strip().lower() for p in lhs.split('[AND]'))
            else:
                premises = (lhs.lower(),)
            conclusion = rhs.strip().lower()
            rules.append((premises, conclusion))
        # Fact
        else:
            parts = clause.split()
            if len(parts) == 2:
                ent, atom = parts[0].lower(), parts[1].lower()
                if entity is None:
                    entity = ent
                elif entity != ent:
                    raise ValueError(f"Multiple entities found: {entity} vs {ent}")
                facts.add(atom)
    return entity, facts, rules, query

In [26]:
def derive_with_explanation(facts, rules, query):
    """
    facts:   set[str]              e.g. {"aggressive","hot"}
    rules:   list[ (tuple[str,...], str) ]
             e.g. [(("difficult","uptight"), "tense"), …]
    query:   str                   e.g. "tense"

    Returns: (label:int, explanation:list[str])
    """
    # --- 1) Forward‐chaining distances & parents ---
    dist   = {a: 0 for a in facts}
    parent = {}   # parent[conclusion] = (rule_index, premises)
    pq     = [(0, a) for a in facts]
    heapq.heapify(pq)

    # Build waiting list for hyperedges
    waiting   = {}   # atom → [rule_idx,…]
    sat_count = [0]*len(rules)
    for i, (prem, conc) in enumerate(rules):
        for p in prem:
            waiting.setdefault(p, []).append(i)

    # Dijkstra‐style forward
    while pq:
        cost_x, x = heapq.heappop(pq)
        if cost_x > dist[x]:
            continue
        if x == query:
            break

        for ri in waiting.get(x, []):
            sat_count[ri] += 1
            prem, conc = rules[ri]
            if sat_count[ri] == len(prem):
                c = max(dist[p] for p in prem) + 1
                if conc not in dist or c < dist[conc]:
                    dist[conc]         = c
                    parent[conc]       = (ri, prem)
                    heapq.heappush(pq, (c, conc))

    def build_proof(atom):
        if atom in facts:
            return []
        ri, prem = parent[atom]
        proof = []
        for p in prem:
            proof += build_proof(p)
        proof.append(f"{' ∧ '.join(prem)} ⇒ {atom}")
        return proof

    # Decide label and explanation in one place
    if query in dist:
        label       = 1
        explanation = build_proof(query)
    else:
        label       = 0
        # build failure explanation as before
        expl = []
        candidate_rules = [
            (i, prem) for i,(prem,conc) in enumerate(rules) if conc==query
        ]
        if not candidate_rules:
            expl.append(f"No rule concludes “{query}.”")
        else:
            for _, prem in candidate_rules:
                available = [p for p in prem if p in dist]
                missing   = [p for p in prem if p not in dist]
                for p in available:
                    expl += build_proof(p)
                expl.append(
                    f"Cannot apply rule “{' ∧ '.join(prem)} ⇒ {query}” "
                    f"because missing: {', '.join(missing)}."
                )
        explanation = expl

    return label, explanation


In [32]:
# Example usage on the alice example
text = (
    "aggressive [IMPLY] hot [PERIOD] "
    "cold [AND] impartial [IMPLY] difficult [PERIOD]"
    "difficult [AND] uptight [IMPLY] tense [PERIOD] "
    "aggressive [IMPLY] impartial [PERIOD] "
    "alice aggressive [PERIOD] alice uptight [PERIOD] "
    "query : alice is tense [PERIOD]"
)

entity, facts, rules, query = parse_logic(text)
label, proof = derive_with_explanation(facts, rules, query)

print("Entity:", entity)
print("Facts:", facts)
print("Rules:", rules)
print("Query:", query)
print("Label =", label)
print("Proof chain:")
for step in proof:
    print(" -", step)

Entity: alice
Facts: {'uptight', 'aggressive'}
Rules: [(('aggressive',), 'hot'), (('cold', 'impartial'), 'difficult'), (('difficult', 'uptight'), 'tense'), (('aggressive',), 'impartial')]
Query: tense
Label = 0
Proof chain:
 - Cannot apply rule “difficult ∧ uptight ⇒ tense” because missing: difficult.


In [23]:
facts = {"aggressive", "hot"}
rules = [
    (("aggressive",), "impartial"),
    (("hot", "impartial"), "difficult"),
    (("difficult",), "tense"),
    (("hot", "aggressive"), "modest"),
    (("modest","tense"), "intelligent")
]
label, proof = derive_with_explanation(facts, rules, "intelligent")
print("Label =", label)

print("Proof chain:")
for step in proof:
    print("  ", step)


Label = 1
Proof chain:
   hot ∧ aggressive ⇒ modest
   aggressive ⇒ impartial
   hot ∧ impartial ⇒ difficult
   difficult ⇒ tense
   modest ∧ tense ⇒ intelligent
